In [1]:
%matplotlib qt

In [1]:
import LadiskDAQ

# Acquisition

Define serial communication settings, make an instance of `acquisition` class:

In [2]:
# an example of incoming data representing one line: 
# b"\xfa\xfb\x00\x00\x00\x01\x00\x00\x00\x02\x00\x03\n"

port = "COM3"      
baudrate = 250000

byte_sequence = (("uint32", 1), ("int32", 1), ("int16", 1)) # sequence of recieved data without starting and ending bytes


acq = LadiskDAQ.SerialAcquisition(port=port, baudrate=baudrate, 
                                  byte_sequence=byte_sequence,
                                  start_bytes=b"\xfa\xfb", # starting bytes (always the same in each line)
                                  end_bytes=b"\n",         # ending bytes (always the same)
                                  write_start_bytes=b"\x01", # bytes to write to data source before acquisition (i.e. start sending data)
                                  write_end_bytes=  b"\x02", # bytes to write to data source after acquisition (i.e. stop sending data)  
                                  sample_rate=500            # approximate sample rate of data recieved. Used to calculate N samples to 
                                                             # capture within specified time interval. TODO: detect this automatically
                                  )
                                  

Set the trigger. If you want the acquisition to start immediately, set the `level` to zero.

In [ ]:
acq.set_trigger(level=0.5, channel=0, duration=11, presamples=10)

# Running the acquisition

The acquisition and generation can be run separately by calling:
```
acq.run_acquisition()
```

To combine include real time plot, put `acq` into the `LDAQ` class. 

In [4]:
# create DAQ object:
ldaq = LadiskDAQ.LDAQ(acq)

# configure plot:
plot_layout={
    (0, 0): [1, 2],           # on subplot[0, 0], plot channels 2 and 3
    (0, 1): [0],              # on subplot[0, 1], plot channel  1
    (1, 0): [(0, 1), (0, 2) ] # on subplot[1, 0], plot channels 2 and 3 versus channel 1
                              # subplot[1, 1] is empty
}

ldaq.configure(plot_layout=plot_layout, 
               max_time=4) # on the subplots, display last 4 seconds of the signal 

The visualization can be configured using the `configure` method. 

The plot layout can be costumized, the acquired data can be processed on-the-fly using a custom function etc. For more details on plot layout, see docstring of the `configure` method.

After the configuration is done, the acquisition can be started by calling the `run` method. Everytime the `run` method is called, the port is opened and data stream is started.

In [5]:
ldaq.run()

+--------+--------------------------------------------------+
| HOTKEY |                   DESCRIPTION                    |
+--------+--------------------------------------------------+
|   q    |               Stop the measurement               |
+--------+--------------------------------------------------+
|   s    | Start the measurement manually (without trigger) |
+--------+--------------------------------------------------+
|   f    |      Freeze the plot during the measurement      |
+--------+--------------------------------------------------+
| Space  |           Resume the plot after freeze           |
+--------+--------------------------------------------------+
	Waiting for trigger...stop.


Measured data can be accessed as `ldaq.acquisition.data`, to save the measurements as dictionary to pickle, use  `ldaq.save_measurement()` method. 


In [9]:
ldaq.acquisition.data

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [4.57075536e+08, 4.40000000e+01, 8.00000000e+01],
       [4.57077532e+08, 4.50000000e+01, 7.90000000e+01],
       [4.57079540e+08, 4.60000000e+01, 7.80000000e+01]])